# Multi Layer Perceptron

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, confusion_matrix


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Carregando os dados

In [ ]:
df_all = pd.read_parquet("../../data/data_processed/participants/Participants_all.parquet")

In [ ]:
# coluna da atividade alvo
TARGET = "label:WillettsSpecific2018"

In [ ]:
label2enc = (
    df_all[["label:WillettsSpecific2018","label:WillettsSpecific2018_enc"]]
    .drop_duplicates()
    .set_index("label:WillettsSpecific2018")["label:WillettsSpecific2018_enc"]
    .sort_index()
    .to_dict()
)

In [ ]:
enc2label = {
    v: k for k, v in label2enc.items()
}

## Preparação do Data Set Para o MLP
Separação de treino e teste

Verficação de Nas

Pre-processamento: one hot-encoding, standard scaling

### Separação em treino e teste

In [ ]:
#features baseadas em aceleração
features_acc = [
    'x_mean', 'x_std','x_min', 'x_max',
    'y_mean', 'y_std', 'y_min', 'y_max',
    'z_mean','z_std', 'z_min', 'z_max',
    'energy_x', 'energy_y', 'energy_z','energy_total',
    'magnitude_mean', 'corr_xy', 'corr_xz', 'corr_yz',
    'fft_dom_freq', 'fft_peak_power'
]


#features de contexto, não baseadas em aceleração.
features_cont = ['sex', 'age_group', 'hour_sin', 'hour_cos']

In [ ]:
cols_corr = ["corr_xy", "corr_xz", "corr_yz"]
df_all[cols_corr] = df_all[cols_corr].fillna(0)

### Conforme decisão na primeira versão do notebook MLP_Model vamos retirar P043 para teste

In [ ]:
df_all_test = df_all[df_all['pid']=='P043'].reset_index(drop=True)
df_all_train = df_all[df_all['pid']!='P043'].reset_index(drop=True)

In [ ]:
#target
y_train = df_all_train['label:WillettsSpecific2018_enc']
y_test = df_all_test['label:WillettsSpecific2018_enc']

#features apenas com variáveis numéricas de aceleração (22 features)
X_acc_train= df_all_train[features_acc]
X_acc_test= df_all_test[features_acc]

#features com variáveis numéricas de aceleração e features de contexto (26 features)
X_all_train = df_all_train[features_acc+features_cont]
X_all_test = df_all_test[features_acc+features_cont]

### Pre-processamento: standard scaling + one hot-encoding

In [ ]:
def build_preprocessor(X_train):
    '''sex não é escalado → mantido como 0/1
    age_group é one-hot
    numeric_features recebem scaler
    ordem das colunas é garantida'''

    # 1. Definir grupos de features
    categorical_features = ["age_group"] if "age_group" in X_train.columns else []

    # variáveis que NÃO devem ser transformadas (mantidas como estão)
    passthrough_features = []
    if "sex" in X_train.columns:
        passthrough_features.append("sex")

    # numéricas contínuas = todas as outras
    numeric_features = [
        col for col in X_train.columns
        if col not in categorical_features + passthrough_features
    ]

    # 2. Transformadores
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(
        handle_unknown="ignore",
        sparse_output=False
    )

    # 3. ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
            ("pass", "passthrough", passthrough_features),
        ],
        remainder="drop"
    )

    # 4. Fit no treino
    preprocessor.fit(X_train)

    # 5. Construção correta dos nomes finais
    # numéricas escaladas
    out_num = numeric_features

    # categorias one-hot
    out_cat = (
        preprocessor.named_transformers_["cat"]
        .get_feature_names_out(categorical_features)
        .tolist()
        if categorical_features else []
    )

    # binárias mantidas (sexo)
    out_pass = passthrough_features

    feature_names = out_num + out_cat + out_pass

    return preprocessor, feature_names

In [ ]:
def apply_preprocessor(preprocessor, X, feature_names):
    X_trans = preprocessor.transform(X)
    return pd.DataFrame(X_trans, columns=feature_names, index=X.index)

### V1 — só aceleração

In [ ]:
prep_v1, feat_v1 = build_preprocessor(X_acc_train)
X_v1_train = apply_preprocessor(prep_v1, X_acc_train, feat_v1)
X_v1_test  = apply_preprocessor(prep_v1, X_acc_test, feat_v1)

### V2 — aceleração + contexto

In [ ]:
prep_v2, feat_v2 = build_preprocessor(X_all_train)
X_v2_train = apply_preprocessor(prep_v2, X_all_train, feat_v2)
X_v2_test  = apply_preprocessor(prep_v2, X_all_test, feat_v2)

## MLP

### Duas possíveis arquiteturas

In [ ]:
#Versão mais robusta para rodar no colab
def build_mlp_baseline(n_features, n_classes):
    model = models.Sequential()

    # Layer 1
    model.add(layers.Dense(256, input_shape=(n_features,), activation=None))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.4))

    # Layer 2
    model.add(layers.Dense(128, activation=None))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))

    # Layer 3
    model.add(layers.Dense(64, activation=None))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.2))

    # Output
    model.add(layers.Dense(n_classes, activation='softmax'))

    return model

In [ ]:
#Versão mais leve para V1
def build_mlp_light(n_features, n_classes):
    model = models.Sequential()

    model.add(layers.Dense(128, input_shape=(n_features,), activation=None))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(64, activation=None))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(32, activation=None))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.1))

    model.add(layers.Dense(n_classes, activation='softmax'))

    return model

### Compilador

In [ ]:
def compile_mlp(model, lr=1e-3):
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

### Critérios a serem usados no fit

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

In [ ]:
class_weight_values = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = {
    cls: weight for cls, weight in zip(np.unique(y_train), class_weight_values)
}

In [ ]:
class_weights

### Rodar no google colab apenas com variáveis de aceleração e arquitetura light

In [ ]:
n_features = X_v1_train.shape[1] #X_v1_train ou X_v2_train
X_train_tensor = X_v1_train.to_numpy().astype(np.float32)

n_classes = 10
y_train_onehot = to_categorical(y_train, num_classes=n_classes)


In [ ]:
model = build_mlp_baseline(n_features, n_classes)  # build_mlp_light ou build_mlp_baseline
model = compile_mlp(model, lr=1e-3)

### Primeiro treino

In [ ]:
history = model.fit(
    X_train_tensor,
    y_train_onehot,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    class_weight=class_weights,
    callbacks=[early_stop],
    shuffle=True,
    verbose=1
)

## Avaliação resultados

### Criação do ypred

In [ ]:
X_test_tensor = X_v1_test.values.astype("float32")
y_test_onehot = to_categorical(y_test, num_classes=10)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_tensor, y_test_onehot, verbose=1)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)

In [ ]:
y_pred_proba = model.predict(X_test_tensor)

In [ ]:
y_pred = y_pred_proba.argmax(axis=1)

### Macro F1

In [ ]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1:", macro_f1)

In [ ]:
f1_per_class = f1_score(y_test, y_pred, average=None)
f1_named = {
    enc2label[i]: f1_per_class[i]
    for i in range(len(f1_per_class))
}
for label, f1_value in sorted(f1_named.items(), key=lambda x: x[1], reverse=True):
    print(f"{label:20s}  F1 = {f1_value:.4f}")

### Matriz de confusão

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
labels_order = [enc2label[i] for i in range(10)]

cm_df = pd.DataFrame(
    cm,
    index=labels_order,
    columns=labels_order
)

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix (P043)")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

### Resumo dos resultados

In [ ]:
summary = {
    "test_accuracy": float(test_accuracy),
    "test_loss": float(test_loss),
    "macro_f1": float(macro_f1),
    "f1_per_class": f1_named,
    "confusion_matrix": cm_df
}

In [ ]:
print("=== Summary for Participant P043 ===\n")

print(f"Test Accuracy:   {summary['test_accuracy']:.4f}")
print(f"Test Loss:       {summary['test_loss']:.4f}")
print(f"Macro F1:        {summary['macro_f1']:.4f}\n")

print("F1 per class:")
for label, value in summary["f1_per_class"].items():
    print(f"  {label:20s} {value:.4f}")

print("\nConfusion Matrix:")
display(summary["confusion_matrix"])